**Normalized coordinates are calculated in each 1m x 1m block.**

**NOT WORKING WELL**

**Read .ply file of point cloud**

In [2]:
import open3d as o3d
import numpy as np
import os
import h5py

In [3]:
script_dir = os.path.dirname(os.getcwd())
sim_pc_path = os.path.join(script_dir, '.', 'docs', 'smartLab_simulated.ply')

pcd = o3d.io.read_point_cloud(sim_pc_path)
points = np.asarray(pcd.points)          # Shape: [N, 3]
colors = np.asarray(pcd.colors)           # Shape: [N, 3]

#o3d.visualization.draw_geometries([pcd])

In [4]:
# Concatenate to get features of shape (N, 9)
features = np.concatenate([points, colors], axis=1)

In [5]:
color_map = np.array([
    [1.0, 0.0, 0.0],  # Label 0: Red,  'ceiling'
    [0.0, 1.0, 0.0],  # Label 1: Green, 'floor'
    [0.0, 0.0, 1.0],  # Label 2: Blue,  'wall'
    [1.0, 1.0, 0.0],  # Label 3: Yellow, 'beam'
    [1.0, 0.0, 1.0],  # Label 4: Magenta, 'column'
    [0.0, 1.0, 1.0],  # Label 5: Cyan, 'window'
    [0.5, 0.5, 0.5],  # Label 6: Gray, 'door'
    [1.0, 0.5, 0.0],  # Label 7: Orange, 'chair'
    [0.5, 0.0, 1.0],  # Label 8: Purple, 'table'
    [0.5, 1.0, 0.5],  # Label 9: Light Green, 'bookcase'
    [0.5, 0.5, 1.0],  # Label 10: Light Blue, 'sofa'
    [1.0, 0.5, 0.5],  # Label 11: Pink, 'board'
    [0.0, 0.0, 0.0]   # Label 12: Black, 'clutter'
    ])

# Find closest color in color_map for each point
#labels = np.argmin(np.linalg.norm(colors[:, None] - color_map, axis=2), axis=1)

**Blocks of 1m x 1m x 1m with 4096 points in each like S3DIS dataset**

Features of point (9):
coordinates(3), colors(3), normalized coordinates(3) (normalization in each block)

In [9]:
min_bound = np.min(points, axis=0)
max_bound = np.max(points, axis=0)

block_size = 1.0  # in meters
num_blocks_x = int(np.ceil((max_bound[0] - min_bound[0]) / block_size))
num_blocks_y = int(np.ceil((max_bound[1] - min_bound[1]) / block_size))
num_blocks_z = int(np.ceil((max_bound[2] - min_bound[2]) / block_size))

block_features_list = []
block_labels_list  = []

for ix in range(num_blocks_x):
    for iy in range(num_blocks_y):
        for iz in range(num_blocks_z):
            # Define the spatial boundaries for this block
            x_min = min_bound[0] + ix * block_size
            x_max = x_min + block_size
            y_min = min_bound[1] + iy * block_size
            y_max = y_min + block_size
            z_min = min_bound[2] + iz * block_size
            z_max = z_min + block_size

            # Find indices of points within the block
            in_block = np.where(
                (points[:, 0] >= x_min) & (points[:, 0] < x_max) &
                (points[:, 1] >= y_min) & (points[:, 1] < y_max) &
                (points[:, 2] >= z_min) & (points[:, 2] < z_max)
            )[0]

            if len(in_block) == 0:
                continue  # Skip empty blocks

            # Retrieve original features (assumed to be [x,y,z,r,g,b])
            block_features_orig = features[in_block, :]
            # Retrieve the spatial coordinates to use for normalization.
            block_coords = points[in_block, :3]
            # Compute normalized coordinates per block: shift by block minimum and scale by block size.
            normalized_coords = (block_coords - np.array([x_min, y_min, z_min])) / block_size

            # Concatenate the original 6 features with the new normalized coordinates (nx, ny, nz).
            block_features = np.concatenate([block_features_orig, normalized_coords], axis=1)
            
            # If label is directly extractable, substitute here. If not, infer from color.
            block_colors = colors[in_block]
            block_labels = np.argmin(np.linalg.norm(block_colors[:, None] - color_map, axis=2), axis=1)

            # --- Handling Block Size (4096 points) ---
            # If there are more than 4096 points, randomly sample 4096.
            # If there are fewer, perform random duplication (or padding with zeros) to reach 4096.
            num_points = block_features.shape[0]
            #print(num_points)
            target_points = 4096

            if num_points >= target_points:
                idx = np.random.choice(num_points, target_points, replace=False)
            else:
                # Duplicate some points
                idx = np.concatenate([
                    np.arange(num_points),
                    np.random.choice(num_points, target_points - num_points, replace=True)
                ])
            block_features = block_features[idx, :]
            block_labels = block_labels[idx]

            block_features_list.append(block_features)
            block_labels_list.append(block_labels)

53112
51267
43738
51438
47284
32229
39209
38237
54267
45716
34676
38518
43356
32521
29659
37598
52703
49766
35671
37982
44717
36658
30995
37453
32489
26372
20259
37034
32130
24344
34424
37658
33020
23704
32417
37720
32890
20238
19950
37449
43483
37839
35242
46130
7873
10940
12172
11954
32374
23476
20589
35401
26511
10660
2487
20069
20042
19891
20110
492
649
20764
19977
1273
882
20975
20017
19859
19961
20003
19780
19803
19892
19986
19878
19925
32923
19801
17410
28139
4578
6643
10129
11319
38147
30315
30408
43977
20100
20133
19871
19988
20051
20053
19957
20071
19983
20173
20075
19909
19911
20078
19807
19866
20050
20257
33033
21640
19832
28117
8546
6071
10012
11178
30498
19883
19728
35422
24690
20052
24084
19740
20069
20269
19895
19929
19682
19822
19968
20067
20177
20035
19990
19991
19952
20056
35166
27003
23644
27792
8908
4373
9809
11209
35348
50526
36996
37227
22789
118
19998
19844
19968
25494
20005
41051
19966
27127
19956
19875
19877
19923
20004
20007
20303
19968
20045
35778
27865
2410

KeyboardInterrupt: 

In [7]:
data_array = np.stack(block_features_list, axis=0)  # Shape: (B, 4096, 9)
label_array = np.stack(block_labels_list, axis=0)     # Shape: (B, 4096)

print(data_array[0][0])
print(label_array[0][0])



[-5.65256509 -6.28034615  0.          0.          0.          1.
  0.16786251  0.4688542   0.35540296]
2


In [8]:
sim_pc_data_path = os.path.join(script_dir, '.', 'docs', 'sim_pc_dataset_normbyblock.h5')

# Save to HDF5
with h5py.File(sim_pc_data_path, 'w') as f:
    f.create_dataset('data', data=data_array, compression='gzip')
    f.create_dataset('label', data=label_array, compression='gzip')

In [9]:
import h5py
f = h5py.File(sim_pc_data_path, 'r') #train_pc_path, data_file_path
list(f.keys())
dset = f['data']
dset.shape

(485, 4096, 9)

In [10]:
from pprint import pprint
pprint(dset[0][2].tolist())

[-5.620427605685707,
 -6.410465555278665,
 -0.0744237747039048,
 0.0,
 1.0,
 0.0,
 0.017368742185738886,
 0.030458044305317365,
 0.07192523227401235]


In [11]:
dlabel = f['label']
dlabel.shape

(485, 4096)